# Pytorch로 딥러닝 제대로 배우기-기초
## Part8: Loss & Optimizer
### 필수 라이브러리 Import

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Compose
from torch.optim import SGD, Adam, RMSprop, Adagrad

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### 1. Baseline 모델 준비

#### (1) 데이터 준비

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# 공개된 데이터 셋으로부터 학습 데이터 다운로드 
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개된 데이터 셋으로부터 테스트 데이터 다운로드
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

#### (2) 모델

In [5]:
class SequentialNetwork(nn.Module):
    def __init__(self):
        super(SequentialNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

#### (3) Optimizer & Losses

In [7]:
# Optimizer
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

# losses
losses = nn.CrossEntropyLoss()

#### (4) 학습 & 테스트 wrapper

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    print("Training mode: {}".format(model.training))
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    print("Training mode: {}".format(model.training))
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### (5) 학습

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 2.315131  [    0/60000]
loss: 0.958521  [ 6400/60000]
loss: 0.545631  [12800/60000]
loss: 0.684434  [19200/60000]
loss: 0.554089  [25600/60000]
loss: 0.515197  [32000/60000]
loss: 0.479842  [38400/60000]
loss: 0.626373  [44800/60000]
loss: 0.564204  [51200/60000]
loss: 0.513132  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 82.6%, Avg loss: 0.486995 

Epoch 2
-------------------------------
Training mode: True
loss: 0.351582  [    0/60000]
loss: 0.435309  [ 6400/60000]
loss: 0.318499  [12800/60000]
loss: 0.485200  [19200/60000]
loss: 0.445094  [25600/60000]
loss: 0.430733  [32000/60000]
loss: 0.376735  [38400/60000]
loss: 0.547977  [44800/60000]
loss: 0.492430  [51200/60000]
loss: 0.499828  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 84.0%, Avg loss: 0.442996 

Epoch 3
-------------------------------
Training mode: True
loss: 0.296853  [    0/60000]
loss: 0.378386  [ 6400/60000]
loss: 0.291324

###  Optimization Function 변경

#### (1) Adam

In [11]:
model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [12]:
# Optimizer
optimizer = Adam(model.parameters(), lr=0.01)

# losses
losses = nn.CrossEntropyLoss()

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 2.309404  [    0/60000]
loss: 0.567684  [ 6400/60000]
loss: 0.455061  [12800/60000]
loss: 0.543201  [19200/60000]
loss: 0.702378  [25600/60000]
loss: 0.565602  [32000/60000]
loss: 0.465732  [38400/60000]
loss: 0.539804  [44800/60000]
loss: 0.588532  [51200/60000]
loss: 0.638516  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 82.1%, Avg loss: 0.504233 

Epoch 2
-------------------------------
Training mode: True
loss: 0.346195  [    0/60000]
loss: 0.397658  [ 6400/60000]
loss: 0.446206  [12800/60000]
loss: 0.548706  [19200/60000]
loss: 0.502140  [25600/60000]
loss: 0.374382  [32000/60000]
loss: 0.462183  [38400/60000]
loss: 0.515804  [44800/60000]
loss: 0.487796  [51200/60000]
loss: 0.448394  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 82.1%, Avg loss: 0.497122 

Epoch 3
-------------------------------
Training mode: True
loss: 0.298856  [    0/60000]
loss: 0.359455  [ 6400/60000]
loss: 0.366930

In [14]:
model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [15]:
# Optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# losses
losses = nn.CrossEntropyLoss()

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 2.318152  [    0/60000]
loss: 0.549940  [ 6400/60000]
loss: 0.406668  [12800/60000]
loss: 0.511418  [19200/60000]
loss: 0.466939  [25600/60000]
loss: 0.443366  [32000/60000]
loss: 0.361802  [38400/60000]
loss: 0.527906  [44800/60000]
loss: 0.491846  [51200/60000]
loss: 0.522291  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 85.6%, Avg loss: 0.410393 

Epoch 2
-------------------------------
Training mode: True
loss: 0.266036  [    0/60000]
loss: 0.350226  [ 6400/60000]
loss: 0.290816  [12800/60000]
loss: 0.393101  [19200/60000]
loss: 0.435963  [25600/60000]
loss: 0.414198  [32000/60000]
loss: 0.321330  [38400/60000]
loss: 0.466281  [44800/60000]
loss: 0.380185  [51200/60000]
loss: 0.438845  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.399133 

Epoch 3
-------------------------------
Training mode: True
loss: 0.247966  [    0/60000]
loss: 0.327447  [ 6400/60000]
loss: 0.221870

#### (2) RMSProp

In [17]:
model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [18]:
# Optimizer
optimizer = RMSprop(model.parameters(), lr=0.01)

# losses
losses = nn.CrossEntropyLoss()

In [19]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 2.310624  [    0/60000]
loss: 0.788663  [ 6400/60000]
loss: 0.656996  [12800/60000]
loss: 0.800855  [19200/60000]
loss: 0.707021  [25600/60000]
loss: 0.439014  [32000/60000]
loss: 0.612473  [38400/60000]
loss: 0.721132  [44800/60000]
loss: 0.756072  [51200/60000]
loss: 0.621178  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 79.2%, Avg loss: 0.552459 

Epoch 2
-------------------------------
Training mode: True
loss: 0.392580  [    0/60000]
loss: 0.464954  [ 6400/60000]
loss: 0.391824  [12800/60000]
loss: 0.450053  [19200/60000]
loss: 0.448481  [25600/60000]
loss: 0.454654  [32000/60000]
loss: 0.551206  [38400/60000]
loss: 0.549378  [44800/60000]
loss: 0.565963  [51200/60000]
loss: 0.489255  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 81.2%, Avg loss: 0.510267 

Epoch 3
-------------------------------
Training mode: True
loss: 0.346622  [    0/60000]
loss: 0.412055  [ 6400/60000]
loss: 0.392066

### Loss Function 변경

#### (1) KLDivLoss

In [20]:
import torch.nn.functional as F

# 공개된 데이터 셋으로부터 학습 데이터 다운로드 
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Compose([
      lambda x: torch.tensor([x]),
      lambda x: F.one_hot(x, 10),
      lambda x: torch.squeeze(x)]),
)

# 공개된 데이터 셋으로부터 테스트 데이터 다운로드
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=Compose([
      lambda x: torch.tensor([x]),
      lambda x: F.one_hot(x, 10),
      lambda x: torch.squeeze(x)])
)

In [21]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [22]:
class SequentialNetwork(nn.Module):
    def __init__(self):
        super(SequentialNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return F.softmax(logits, dim=1)

In [23]:
model = SequentialNetwork().to(device)
model

SequentialNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [24]:
# Optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# losses
losses = nn.KLDivLoss(log_target=True, reduction="batchmean")

In [25]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    print("Training mode: {}".format(model.training))
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # y = torch.squeeze(y)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred.log(), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    print("Training mode: {}".format(model.training))
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            # y = torch.squeeze(y)

            pred = model(X)
            test_loss += loss_fn(pred.log(), y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, losses, optimizer)
    test(test_dataloader, model, losses)
print("Done!")

Epoch 1
-------------------------------
Training mode: True
loss: 29.711113  [    0/60000]
loss: 29.111412  [ 6400/60000]
loss: 29.031303  [12800/60000]
loss: 29.074701  [19200/60000]
loss: 29.059963  [25600/60000]
loss: 29.051739  [32000/60000]
loss: 29.020842  [38400/60000]
loss: 29.081587  [44800/60000]
loss: 29.052654  [51200/60000]
loss: 29.047750  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 85.2%, Avg loss: 29.037873 

Epoch 2
-------------------------------
Training mode: True
loss: 28.983986  [    0/60000]
loss: 28.985472  [ 6400/60000]
loss: 28.983324  [12800/60000]
loss: 29.019779  [19200/60000]
loss: 29.031984  [25600/60000]
loss: 29.031803  [32000/60000]
loss: 28.988228  [38400/60000]
loss: 29.048679  [44800/60000]
loss: 29.009411  [51200/60000]
loss: 29.013920  [57600/60000]
Training mode: False
Test Error: 
 Accuracy: 86.1%, Avg loss: 29.017618 

Epoch 3
-------------------------------
Training mode: True
loss: 28.957888  [    0/60000]
loss: 28.983799  [ 64